In [1]:
### WARNING: This notebook only has to be run once, to populate Delivery Rate scalars in the database.

import random
import pandas as pd

import psycopg2 as pg
import sqlalchemy


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams.update({'font.size': 24})
plt.rcParams.update({'font.weight': 'bold'})

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

In [2]:
from sqlalchemy import create_engine

POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres'
POSTGRES_PASSWORD = 'postgres'
POSTGRES_DBNAME = 'mam'

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

# Select first 5 rows
pd.read_sql_query('''SELECT * FROM scalars LIMIT 5;''', cnx)

,node_amount,area_sq_m,lpn_fn_ratio,relay_type,delta_ms,speed_ms,series,rep,module,name,type,value,full_row,created_at
0,50,1000,8,MAMRELAY,2,14,5,0,MAM_n50_a1000m2_lpnFn8_5.host_fn_rly_6.wlan[0]...,packetDropQueueOverlow:sum(packetBytes),scalar,0.0,MAM_n50_a1000m2_lpnFn8_5-MAMRELAY-5-20201226-1...,2020-12-28
1,50,800,8,MAMRELAY,5000,2,9,0,MAM_n50_a800m2_lpnFn8_9.host_fn_rly_4.app[0],packets sent,scalar,0.0,MAM_n50_a800m2_lpnFn8_9-MAMRELAY-27-20201225-0...,2020-12-28
2,50,800,8,MAMRELAY,5000,2,9,0,MAM_n50_a800m2_lpnFn8_9.host_fn_rly_4.app[0],packets received,scalar,0.0,MAM_n50_a800m2_lpnFn8_9-MAMRELAY-27-20201225-0...,2020-12-28
3,50,800,8,MAMRELAY,5000,2,9,0,MAM_n50_a800m2_lpnFn8_9.host_fn_rly_4.app[0],data packets sent,scalar,155.0,MAM_n50_a800m2_lpnFn8_9-MAMRELAY-27-20201225-0...,2020-12-28
4,50,1000,8,MAMRELAY,2,14,5,0,MAM_n50_a1000m2_lpnFn8_5.host_fn_rly_6.wlan[0]...,packetDropRetryLimitReached:count,scalar,71.0,MAM_n50_a1000m2_lpnFn8_5-MAMRELAY-5-20201226-1...,2020-12-28


In [3]:
primaryKeyColumns = ['node_amount', 'area_sq_m', 'lpn_fn_ratio', 'relay_type', 'delta_ms', 'speed_ms', 'series', 'rep']
primaryKeyColumnsCSV = ','.join(primaryKeyColumns)
tPrimaryKeyColumnsCSV = ','.join(['t.' + x for x in primaryKeyColumns])

allRuns = pd.read_sql_query('''
WITH RECURSIVE
     t AS (
  select ''' + primaryKeyColumnsCSV + ''' from
    (select ''' + primaryKeyColumnsCSV + ''' from scalars
    order by ''' + primaryKeyColumnsCSV + ''' limit 1) asdf
  union all
  select '''
     +
     ','.join(['(select ' + x + ' from scalars where (' + primaryKeyColumnsCSV + ') > (' + tPrimaryKeyColumnsCSV + ') order by ' + primaryKeyColumnsCSV + ' limit 1)' for x in primaryKeyColumns])
     +                       
  '''from t where t.''' + primaryKeyColumns[0] + ''' is not null)
select * from t where t.''' + primaryKeyColumns[0] + ''' is not null;
''', cnx)
allRuns

,node_amount,area_sq_m,lpn_fn_ratio,relay_type,delta_ms,speed_ms,series,rep
0,50,400,4,BMRELAY,0,2,0,0
1,50,400,4,BMRELAY,0,2,1,0
2,50,400,4,BMRELAY,0,2,2,0
3,50,400,4,BMRELAY,0,2,3,0
4,50,400,4,BMRELAY,0,2,4,0
...,...,...,...,...,...,...,...,...
11067,200,1000,12,MAMRELAY,20000,14,5,0
11068,200,1000,12,MAMRELAY,20000,14,6,0
11069,200,1000,12,MAMRELAY,20000,14,7,0
11070,200,1000,12,MAMRELAY,20000,14,8,0


In [71]:
moduleName = 'MAM-Large-Generate-Delivery-Rate-Scalars'
simCount = len(allRuns)
progress = 0

def saveDeliveryRate(data, deliveryRate):
    formattedValues = ''
    for x in primaryKeyColumns:
        formattedField = data[x]
        if (x in ['relay_type']):
            formattedField = '\'' + formattedField + '\''
        formattedValues = formattedValues + str(formattedField) + ','
    formattedValues = formattedValues + '\'' + moduleName + '\','
    
    formattedValues = formattedValues + '\'delivery rate\','
    formattedValues = formattedValues + '\'' + str(deliveryRate) + '\''
    
    query = 'insert into scalars (' + primaryKeyColumnsCSV + ',module,name,value,created_at) ' + '''
        values (''' + formattedValues + ', now()) on conflict do nothing;'
    #print(query)
    cnx.execute(query)

while progress < simCount:
    temp = allRuns.iloc[progress]

    testa = pd.read_sql_query('select name, value from scalars where node_amount=' + str(temp['node_amount']) + 
    ' and area_sq_m=' + str(temp['area_sq_m']) + ' and lpn_fn_ratio='+ str(temp['lpn_fn_ratio']) +
    ' and relay_type=\'' + str(temp['relay_type']) + '\' and delta_ms='+ str(temp['delta_ms']) + 
    ' and speed_ms=' + str(temp['speed_ms']) + ' and series='+ str(temp['series']) + ' and rep=' + str(temp['rep']) + 
    ''' and (name like 'generated packet uuids%' or name like 'received packet uuids%') ''', cnx)
    generatedDf = testa[testa['name'].str.startswith('generated packet uuids')]
    receivedDf = testa[testa['name'].str.startswith('received packet uuids')]

    generatedArr = generatedDf['value']
    receivedArr = receivedDf['value']

    generated = set()
    received = set()

    for x in generatedArr:
        generated.update(x.split(','))

    for x in receivedArr:
        received.update(x.split(','))

    deliveryRate = len(received)/len(generated)*100
    saveDeliveryRate(temp, deliveryRate)
    progress = progress + 1
    if progress % 50 == 0 or (simCount - progress) < 10:
        print('Progress=' + "{:.3f}".format((progress/simCount)*100.0) + '%')


Progress=0.452%
Progress=0.903%
Progress=1.355%
Progress=1.806%
Progress=2.258%
Progress=2.710%
Progress=3.161%
Progress=3.613%
Progress=4.064%
Progress=4.516%
Progress=4.967%
Progress=5.419%
Progress=5.871%
Progress=6.322%
Progress=6.774%
Progress=7.225%
Progress=7.677%
Progress=8.129%
Progress=8.580%
Progress=9.032%
Progress=9.483%
Progress=9.935%
Progress=10.387%
Progress=10.838%
Progress=11.290%
Progress=11.741%
Progress=12.193%
Progress=12.645%
Progress=13.096%
Progress=13.548%
Progress=13.999%
Progress=14.451%
Progress=14.902%
Progress=15.354%
Progress=15.806%
Progress=16.257%
Progress=16.709%
Progress=17.160%
Progress=17.612%
Progress=18.064%
Progress=18.515%
Progress=18.967%
Progress=19.418%
Progress=19.870%
Progress=20.322%
Progress=20.773%
Progress=21.225%
Progress=21.676%
Progress=22.128%
Progress=22.579%
Progress=23.031%
Progress=23.483%
Progress=23.934%
Progress=24.386%
Progress=24.837%
Progress=25.289%
Progress=25.741%
Progress=26.192%
Progress=26.644%
Progress=27.095%
Pr

In [79]:
lastRun = allRuns.iloc[len(allRuns)-1]
lastRunDeliveryRate = pd.read_sql_query('select * from scalars where node_amount=' + str(lastRun['node_amount']) + 
' and area_sq_m=' + str(lastRun['area_sq_m']) + ' and lpn_fn_ratio='+ str(lastRun['lpn_fn_ratio']) +
' and relay_type=\'' + str(lastRun['relay_type']) + '\' and delta_ms='+ str(lastRun['delta_ms']) + 
' and speed_ms=' + str(lastRun['speed_ms']) + ' and series='+ str(lastRun['series']) + ' and rep=' + str(lastRun['rep']) + 
''' and (name = 'delivery rate') ''', cnx)

if lastRunDeliveryRate.empty:
    raise Exception('Non-existent delivery rate. Make sure to have run MAM-Large-Generate-Delivery-Rate-Scalars notebook')

lastRunDeliveryRate

,node_amount,area_sq_m,lpn_fn_ratio,relay_type,delta_ms,speed_ms,series,rep,module,name,type,value,full_row,created_at
0,200,1000,12,MAMRELAY,20000,14,9,0,MAM-Large-Generate-Delivery-Rate-Scalars,delivery rate,None,9.113396693138915,None,2021-02-04
